In [ ]:
#Librerias:

from bs4 import BeautifulSoup
import requests
from deep_translator import GoogleTranslator
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os

In [ ]:
#Traer la página on bs4 para extraer los enlaces:

origin = 'https://www.classcentral.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
response = requests.get(origin, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

## Se buscan los enlaces
url_original = []
for link in soup.body.find_all("a"):
    enlace = link.get("href")
    if enlace is not None:
      url_original.append(enlace)

In [ ]:
len(url_original)

In [ ]:
#Se organizan los enlaces
i = 0
url_todos = []
for enlace in url_original:
  if enlace.startswith("/"):
    nueva_url = "https://www.classcentral.com" + enlace
    url_todos.append(nueva_url)
  i+=1

In [ ]:
len(url_todos)

In [ ]:
#Se toman los enlaces unicos
url_unique = []
for i in url_todos:
  if i in url_unique:
    continue
  else:
    url_unique.append(i)

In [ ]:
#Se crea diccionario para facilitar la busqueda del enlace y a qué corresponde
enlaces = {}
for enlace in url_unique:
  valor = enlace
  dic = enlace.replace("https://www.classcentral.com/", "")
  if dic == '':
    dic = "classcentral"
    enlaces[dic] = enlace
  else:
    enlaces[dic] = enlace

In [ ]:
enlaces

In [ ]:
"""for i in enlaces:
    if 'https://www.classcentral.com/cdn-cgi/l/email-protection#bc83cfc9ded6d9dfc8819addd1cc87ded3d8c581998e8cd4c8c8cccf998ffd998efa998efacbcbcb92dfd0ddcfcfdfd9d2c8ceddd092dfd3d1998efa' == enlaces[i]:
        enlaces.pop(i)"""

In [ ]:
enlaces2 = {}
for enlace in url_unique:
  dic = enlace.replace("https://www.classcentral.com/", "")
  if dic == '':
    dic = "classcentral"
    enlaces2[dic.replace("/","_")] = enlace
  else:
    enlaces2[dic.replace("/","_")] = enlace

In [ ]:
dic3 = {}
for enlace in url_original:
  value = enlace
  if enlace.startswith("/"):
      enlace = enlace.replace("/","")
  dic = enlace.replace("https://www.classcentral.com/", "")
  
  if dic == '':
    dic = "classcentral.html"
    dic3[dic.replace("/","_")] = value
  else:
    dic = dic + ".html"
    dic3[dic.replace("/","_")] = value

In [ ]:
#Traducir y descargar todos los html:
j = 0
driver = webdriver.Chrome()
try:
  for nombre in enlaces2:
      name = nombre + ".html"
      for i in enlaces:
        driver.get(enlaces[i])
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        ## Copiar el código fuente de la página
        html = driver.page_source
        
        #Transformar para capturar datos del html
        soup = BeautifulSoup(html, 'html.parser')

        traductor = GoogleTranslator(source='en', target='hi')
        trans = {}
        trans_dir = {}

        #Obtener texto traducido
        for child in soup.stripped_strings:
          if len(child) > 5000:
            continue
          else:
            if child == 'Copy link':
              continue
            else:
              traduccion = (traductor.translate(child))
              trans[str(child)]=str(traduccion)

        #Valores unicos para traducir      
        texto = list(soup.stripped_strings)
        texto2 = []
        for palabra in texto:
            if palabra in texto2:
                continue
            else:
                texto2.append(palabra)
        #traduccion de la pagina
        for l in range (len(texto2)):
            for texto in soup.find_all(string=True):
              if texto.parent.name != 'style' and texto.parent.name != 'script':
                if texto2[l] in trans.keys():
                    nuevo_texto = texto.replace(texto2[l], trans[texto2[l]])
                    texto.replace_with(nuevo_texto)

        #Cambiar enlaces:
        links = soup.body.find_all("a")

        for s in dic3:
          for link in links:
            a = link.get('href')
            if a == dic3[s]:
              link['href'] = s
          

        #descargar archivo traducido:
        mensaje = str(soup)
        

        archivo = open(name, "w", encoding="utf-8")
        archivo.write(mensaje)
        archivo.close()
        j+=1
except NameError:
    print(name, NameError)

driver.close()

In [ ]:
"""        links = soup.body.find_all("a")     
        for llave in enlaces:
          for link in links:
            if links[link] == enlaces[llave]:
              new = llave + ".html"
              links['link'] = new"""

In [ ]:
"""links = soup.body.find_all("a")

for i in url_original:
    for link in links:
        a = link.get('href')
        if a == i:
            for s in dic3:
                if i == dic3[s]:
                    print(a)"""

In [ ]:
"""links = soup.body.find_all("a")

for s in dic3:
    for link in links:
        a = link.get('href')
        if a == dic3[s]:
            link['href'] = s"""